# **Paul Golder - Capstone Project March 2025**

## Objectives

* This notebook is a scratch pad for code to be used to create visualizations in either python or using python in powerbi

## Inputs

* files from the ETL and ML_Model_Chatgpt_refactor files

## Outputs

* Code or images for inclusion in the dashboard.

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

In [2]:
import os
current_dir = os.getcwd()
current_dir

'h:\\VScode\\CapStoneProject_2025-1\\f_dashboard'

In [3]:
# check current directory contains the file README.md
if os.path.exists("README.md"):
    print("The file README.md exists in the current directory")
else:
    print("The file README.md does not exist in the current directory")
    print("You are in the directory: ", current_dir)
    print("Changing current directory to its parent directory")
    os.chdir(os.path.dirname(current_dir))
    print("You set a new current directory")
    current_dir = os.getcwd()
    if os.path.exists("README.md"):
        print("The file README.md exists in the current directory")
    else:
        RuntimeError("The file README.md does not exist in the current directory, please check the current directory")
        print("Current Directory =", current_dir)

The file README.md does not exist in the current directory
You are in the directory:  h:\VScode\CapStoneProject_2025-1\f_dashboard
Changing current directory to its parent directory
You set a new current directory
The file README.md exists in the current directory


# Section 1

Section 1 content

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [25]:
df = pd.read_csv("data//combined_data.zip",
                 usecols={
    'index',
    'title',
    'label',
    'month',
    'day',
    'year',
    'date_clean',
    'article_id',
                 },  
                 dtype = {
    'index': 'int64',
    'title': 'object',
    'label': 'int64',
    'month': 'float64',
    'day': 'float64',
    'year': 'int64',
    'date_clean': 'object',
    'article_id': 'int64',
    },
                 compression='zip')
print("The shape of the data is: ", df.shape)
print(df.info())
df.head()
# filter out rows with dates earlier than 2014-01-01
df['date'] = pd.to_datetime(df['date_clean'])
df = df[df['date'] >= '2014-01-01']
print("The shape of the data is: ", df.shape)
df.head()

# load locationsfromarticle.zip file
locationsfromarticle = (
    pd.read_csv("data//locationsfromarticle.zip",
                usecols={
                    'article_id',
                    'location',
                },
                dtype={
                    'article_id': 'int64',
                    'location': 'object',
                },
                compression='zip')
                )
print(locationsfromarticle.info())
locationsfromarticle.head()
# load unique_locations.csv file
locations = pd.read_csv("data//unique_locations.csv",
                        dtype={
                            'location': 'object',
                            'latitude': 'float64',
                            'longitude': 'float64',
                            'state': 'object',
                            'country': 'object',
                            'continent': 'object',
                            'subcontinent': 'object',
                            'ignore': 'int64'
                            },
                            )
print(locations.info())
locations.head()

# rename Continent to continent and Sub Continent to subcontinent
# locations = locations.rename(columns={'Continent': 'continent', 'Sub Continent': 'subcontinent'}, inplace=True)
print(locations.info())
locations.head()

# set all null subcontinent values to continent value
locations['subcontinent'] = locations['subcontinent'].fillna(locations['continent'])
# set null country values to subcontinent value
locations['country'] = locations['country'].fillna(locations['subcontinent'])
# set null state values to country value
locations['state'] = locations['state'].fillna(locations['country'])

# match locations to locationsfromarticle
locations['location'] = locations['location'].str.lower()
locationsfromarticle['location'] = locationsfromarticle['location'].str.lower()
locationsmerged = locations.merge(locationsfromarticle, on='location', how='left')
print(locationsmerged.info())
locationsmerged.head()
del locations, locationsfromarticle

# drop all rows with ignore = 1
locationsmerged = locationsmerged[locationsmerged['ignore'] != 1]
locationsmerged = locationsmerged.drop(columns=['ignore'])
locationsmerged.head()
locationsmerged.info()

# merge locationsmerged with df only keep rows with a match
locationgraphdf = df.merge(locationsmerged, on='article_id', how='left')
print(locationgraphdf.info())
locationgraphdf.head()

# using plotly create a map that displays the number of fake articles, real articles, and total articles per country


# create a new dataframe with the number of fake articles per country, continent, and subcontinent
fakearticles = locationgraphdf[locationgraphdf['label'] == 1]
fakearticles = fakearticles.groupby(['year', 'month', 'country', 'continent', 'subcontinent']).size().reset_index(name='fake_count')

# create a new dataframe with the number of real articles per country, continent, and subcontinent
realarticles = locationgraphdf[locationgraphdf['label'] == 0]
realarticles = realarticles.groupby(['year', 'month', 'country', 'continent', 'subcontinent']).size().reset_index(name='real_count')

# merge fake and real articles dataframes
articles = pd.merge(fakearticles, realarticles, on=['year', 'month', 'country', 'continent', 'subcontinent'], how='outer').fillna(0)
articles = articles.sort_values(by=['year', 'month', 'country', 'continent', 'subcontinent'])
articles.head()

# save aticles dataframe as data//articlesformap.csv
articles.to_csv("data//articlesformap.csv", index=False)

print("The shape of the data is: ", df.shape)
df.head()
# filter out rows with dates earlier than 2014-01-01
df['date'] = pd.to_datetime(df['date_clean'])
df = df[df['date'] >= '2014-01-01']
print("The shape of the data is: ", df.shape)
df.head()

# load locationsfromarticle.zip file
locationsfromarticle = pd.read_csv("data//locationsfromarticle.zip", compression='zip')
print(locationsfromarticle.info())
locationsfromarticle.head()
# load unique_locations.csv file
locations = pd.read_csv("data//unique_locations.csv")
print(locations.info())
locations.head()

# match locations to locationsfromarticle
locations['location'] = locations['location'].str.lower()
locationsfromarticle['location'] = locationsfromarticle['location'].str.lower()
locationsmerged = locations.merge(locationsfromarticle, on='location', how='left')
print(locationsmerged.info())
locationsmerged.head()
del locations, locationsfromarticle

# drop all rows with ignore = 1
locationsmerged = locationsmerged[locationsmerged['ignore'] != 1]
locationsmerged = locationsmerged.drop(columns=['ignore'])
locationsmerged.head()
locationsmerged.info()

# set article_id as int64 on locationsmerged and df
df['article_id'] = df['article_id'].astype(int)
locationsmerged['article_id'] = locationsmerged['article_id'].astype(int)

# merge locationsmerged with df only keep rows with a match
df['article_id'] = df['article_id'].astype(int)
locationgraphdf = df.merge(locationsmerged, on='article_id', how='left')
print(locationgraphdf.info())
locationgraphdf.head()

# using plotly create a map that displays the number of fake articles, real articles, and total articles per country


# create a new dataframe with the number of fake articles per country, continent, and subcontinent
fakearticles = locationgraphdf[locationgraphdf['label'] == 1]
fakearticles = fakearticles.groupby(['year', 'month', 'state', 'country', 'continent', 'subcontinent']).size().reset_index(name='fake_count')

# create a new dataframe with the number of real articles per country, continent, and subcontinent
realarticles = locationgraphdf[locationgraphdf['label'] == 0]
realarticles = realarticles.groupby(['year', 'month', 'state', 'country', 'continent', 'subcontinent']).size().reset_index(name='real_count')

# merge fake and real articles dataframes
articles = pd.merge(fakearticles, realarticles, on=['year', 'month', 'state', 'country', 'continent', 'subcontinent'], how='outer').fillna(0)
articles = articles.sort_values(by=['year', 'month', 'state', 'country', 'continent', 'subcontinent'])
articles.head()

# save aticles dataframe as data//articlesformap.csv
articles.to_csv("data//articlesformap.csv", index=False)

The shape of the data is:  (113678, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113678 entries, 0 to 113677
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   index       113678 non-null  int64  
 1   title       113678 non-null  object 
 2   label       113678 non-null  int64  
 3   month       38603 non-null   float64
 4   day         113637 non-null  float64
 5   year        113678 non-null  int64  
 6   date_clean  113677 non-null  object 
 7   article_id  113678 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 6.9+ MB
None
The shape of the data is:  (38634, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217999 entries, 0 to 217998
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article_id  217999 non-null  int64 
 1   location    210129 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.3+ MB
No

In [26]:
# Load the articlesformap.csv file
articles = pd.read_csv("data//articlesformap.csv")
articles.head()

# create a map that displays the number of fake articles and real articles per country, continent, and subcontinent
fig = go.Figure()

# Add a slider to filter by month and year
years = articles['year'].unique()
months = articles['month'].unique()

for year in years:
    for month in months:
        filtered_data = articles[(articles['year'] == year) & (articles['month'] == month)]
        fig.add_trace(go.Choropleth(
            visible=False,
            locations=filtered_data['country'],
            z=filtered_data['fake_count'],
            locationmode='country names',
            colorscale='Reds',
            colorbar_title="Number of Fake Articles",
            text=filtered_data[['fake_count', 'real_count', 'continent', 'subcontinent']],
            hovertemplate='<b>%{location}</b><br>Fake Articles: %{z}<br>Real Articles: %{text[1]}<br>Continent: %{text[2]}<br>Subcontinent: %{text[3]}',
            name=f'{year}-{month} (Fake)'
        ))
        fig.add_trace(go.Choropleth(
            visible=False,
            locations=filtered_data['country'],
            z=filtered_data['real_count'],
            locationmode='country names',
            colorscale='Blues',
            colorbar_title="Number of Real Articles",
            text=filtered_data[['fake_count', 'real_count', 'continent', 'subcontinent']],
            hovertemplate='<b>%{location}</b><br>Fake Articles: %{text[0]}<br>Real Articles: %{z}<br>Continent: %{text[2]}<br>Subcontinent: %{text[3]}',
            name=f'{year}-{month} (Real)'
        ))

# Make the first trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(0, len(fig.data), 2):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": f"Mentions per Country - {fig.data[i].name.split(' ')[0]}"}],
        label=fig.data[i].name.split(' ')[0]  # Add label here
    )
    step["args"][0]["visible"][i:i+2] = [True, True]
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Date: "},
    pad={"t": 50},
    steps=steps
)]

# Add filter buttons
button_layer_1_height = 1.12
fig.update_layout(
    sliders=sliders,
    title_text='Article Mentions per Country, Continent, and Subcontinent',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["visible", [True if "Fake" in trace.name else False for trace in fig.data]],
                    label="Fake Article Mentions",
                    method="restyle"
                ),
                dict(
                    args=["visible", [True if "Real" in trace.name else False for trace in fig.data]],
                    label="Real Article Mentions",
                    method="restyle"
                ),
                dict(
                    args=["visible", [True] * len(fig.data)],
                    label="All",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.17,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
    ]
)

fig.show()


---